# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath="../output_data/cities.csv"
cities_df=pd.read_csv(filepath)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.15,-70.92,41.00,86,90,14.99,CL,1603022154
1,Mataura,-46.19,168.86,50.00,77,100,3.00,NZ,1603022154
2,Nikolskoye,59.70,30.79,39.20,70,90,8.95,RU,1603022154
3,Butaritari,3.07,172.79,81.34,78,60,18.95,KI,1603022155
4,Albany,42.60,-73.97,35.01,100,20,3.40,US,1603022107


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations=cities_df[['Lat','Lng']]
weights=cities_df['Humidity']
# mid_map is the Lat and Lng coordinates so I could center the map on the screen.
mid_map=(0,15)
fig=gmaps.figure(center=mid_map,zoom_level=1)
heat_layer=gmaps.heatmap_layer(locations,weights=weights)
heat_layer.max_intensity=100
heat_layer.point_radius=5
heat_layer.dissipation=False
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# I included humidity level as well as the temp, cloudiness and windspeed.
hitemp=80
lwtemp=70
humidity=60
cloudiness=25
windspeed=10
my_spots_df=cities_df.loc[
                    (cities_df['Max Temp']>=lwtemp) & (cities_df['Max Temp']<=hitemp) &
                    (cities_df['Humidity']<=humidity) &
                    (cities_df['Cloudiness']<=cloudiness) & 
                    (cities_df['Wind Speed']<=windspeed) 
                    ,:]
my_spots_df              

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
122,Birjand,32.87,59.22,77.00,3,0,7.20,IR,1603022164
172,Iki-Burul,45.82,44.65,77.00,31,0,8.95,RU,1603022168
197,Xaçmaz,41.46,48.81,75.25,35,0,6.51,AZ,1603022170
206,San Jose,37.34,-121.89,71.01,42,1,2.80,US,1603022043
233,Grand Gaube,-20.01,57.66,79.00,58,1,10.00,MU,1603022173
471,Mingora,34.78,72.36,74.16,25,0,3.71,PK,1603022194
508,Colares,38.80,-9.45,70.00,52,20,9.17,PT,1603022197


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Limit lookups to 10 vacation spots.
lookup_limit=10
hotel_df=my_spots_df.copy()
hotel_df['Hotel Name']=''

params={
        "query":'hotel',
        "radius":5000,
        "type":"lodging",
        "language":"en",
        "key":g_key
}

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
x=0
for index, row in hotel_df.iterrows():
    if (x==lookup_limit):
        break
    x+=1
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    hotel_names=requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_names["results"][0]["name"]
    except:
        hotel_df.loc[index,"Hotel Name"]="No Hotel"
hotel_df.head(lookup_limit)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
122,Birjand,32.87,59.22,77.00,3,0,7.20,IR,1603022164,Birjand Tourist Hotel
172,Iki-Burul,45.82,44.65,77.00,31,0,8.95,RU,1603022168,No Hotel
197,Xaçmaz,41.46,48.81,75.25,35,0,6.51,AZ,1603022170,SAHİL HOTEL
206,San Jose,37.34,-121.89,71.01,42,1,2.80,US,1603022043,The Westin San Jose
233,Grand Gaube,-20.01,57.66,79.00,58,1,10.00,MU,1603022173,Veranda Paul et Virginie Hotel & Spa
471,Mingora,34.78,72.36,74.16,25,0,3.71,PK,1603022194,Burj Al Swat
508,Colares,38.80,-9.45,70.00,52,20,9.17,PT,1603022197,Arribas Sintra Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))